<a href="https://colab.research.google.com/github/mmsamiei/thesis-prototype/blob/master/phase_3/tokenized_train_bert_bert_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/drive/My\ Drive/Datasets/wizard_of_wikipedia/train.json ./train.json

In [3]:
!pip install transformers

     |████████████████████████████████| 450kB 4.8MB/s 
     |████████████████████████████████| 1.0MB 46.5MB/s 
     |████████████████████████████████| 870kB 35.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=4851dd519f130d05907a71a63dfcf4901563cc98b3dcc3cbaab8cb977736855e
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [4]:
import torch
from transformers import BertTokenizer, GPT2Tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [0]:
import json

file = open('./train.json')
data = json.load(file)

In [6]:
!pip install -U tqdm

     |████████████████████████████████| 61kB 2.8MB/s 
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
def bert_text_to_idslist(text):
  '''
  bert tokenizer text -> [CLS] text [SEP]
  '''
  text = str(text)
  text = "[CLS] " + text + " [SEP]"
  return bert_tokenizer.convert_tokens_to_ids(bert_tokenizer.tokenize(text))

In [0]:
def gpt_text_to_idslist(text):
  '''
  bert tokenizer text -> [CLS] text [SEP]
  '''
  text = str(text)
  text = "[CLS] " + text + " [SEP]"
  return gpt_tokenizer.convert_tokens_to_ids(gpt_tokenizer.tokenize(text))

In [14]:
from tqdm.notebook import tqdm

new_dataset = []

for story in tqdm(data):
    dialog = story['dialog']
    history = []
    for turn in dialog:
        if("checked_sentence" in turn and len(history) != 0): ## means he is wizard
            checked_sentence_tokenized = bert_text_to_idslist(turn['checked_sentence'])
            new_row = {}
            new_row['history'] = history.copy()
            new_row['knowledge'] = checked_sentence_tokenized
            new_row['response'] = gpt_text_to_idslist(turn['text'])
            new_dataset.append(new_row)
        tokened_next = bert_text_to_idslist(turn['text'])
        if(len(history)==0):
          history.extend(tokened_next)
        else:
          history.extend(tokened_next[1:])

In [19]:
len(new_dataset)

74092

In [25]:
gpt_tokenizer.decode(new_dataset[100]['response'])

'[CLS] The word "piano" means soft, and the piano can be played different volumes by using different pressure on the keys. [SEP]'

In [24]:
bert_tokenizer.decode(new_dataset[100]['knowledge'])

'[CLS] {\'chosen _ piano _ 0\':\'the italian musical terms " piano " and " forte " indicate " soft " and " loud " respectively, in this context referring to the variations in volume ( i. e., loudness ) produced in response to a pianist \\\'s touch or pressure on the keys : the greater the velocity of a key press, the greater the force of the hammer hitting the strings, and the louder the sound of the note produced and the stronger the attack.\'} [SEP]'

In [26]:
bert_tokenizer.decode(new_dataset[100]['history'])

"[CLS] what do you know about the piano? great instrument! [SEP] it has been around since 1700. [SEP] where was it invented, do you know? [SEP] it was invented in italy by bartolomeo cristofori. [SEP] interesting! what else can you tell me please? i'm so curious. [SEP]"

In [0]:
with open('tokenized_phase3.json', 'w') as outfile:
  json.dump(new_dataset, outfile)

In [0]:
!cp /content/tokenized_phase3.json /content/drive/My\ Drive/Thesis/phase-3